<h2>0 - Setup do sistema</h2>

<h3 style="font-style:italic"><b>0.2 - Importação dos módulos</b></h3>

In [ ]:
import pandas as pd
import decimal
import numpy as np
from tabulate import tabulate
from fuzzywuzzy import process
import gspread
import tweepy 
from tweepy import OAuthHandler
import time
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import sys
import argparse
import json
import logging
import codecs
import os.path
from os.path import join as pjoin
import glob
import json
from io import StringIO
from io import open
from urllib.request import urlopen
from itertools import chain
import os
import csv
import regex as re
from csv import DictReader
import itertools
import networkx as nx
from networkx.algorithms.community import girvan_newman
from networkx.algorithms.community import greedy_modularity_communities
from networkx.readwrite import json_graph
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sys
import os.path
import time
import argparse
import json
import tweepy
from tweepy import OAuthHandler
import logging
import codecs
from os.path import join as pjoin


<h3 style="font-style:italic">0.4 - Declaração das chaves de API do Twitter</h3>

<p>É preciso requisitá-las na <a href="https://developer.twitter.com/en/portal/petition/academic/is-it-right-for-you">página de desenvolvedores da plataforma.</a></p>

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret) 
api = tweepy.API(auth) 

<h3 style="font-style:italic">0.5 - Download dos dados da Câmara dos Deputados e do Tribunal Superior Eleitoral</h3>

<table style="width:100%;border: 1px solid black; border-collapse: collapse;">
  <tr>
    <th style="width:100%;border: 1px solid black; border-collapse: collapse">Câmara</th>
    <th style="width:100%;border: 1px solid black; border-collapse: collapse">TSE</th> 
  </tr>
  <tr>
      <td style="width:100%;border: 1px solid black; border-collapse: collapse"><a href="https://dadosabertos.camara.leg.br/arquivos/deputados/csv/deputados.csv">deputados</a></td>
    <td style="width:100%;border: 1px solid black; border-collapse: collapse"><a href="https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2018.zip">candidatos2018</a></td>
  </tr>
   
  <tr>
    <td style="width:100%;border: 1px solid black; border-collapse: collapse"><a href="https://dadosabertos.camara.leg.br/arquivos/proposicoes/csv/proposicoes-2020.csv">proposicoes</a></td>  
    <td style="width:100%;border: 1px solid black; border-collapse: collapse"><a href="https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2018.zip">bensCandidatos2018</a></td>
    </tr>
    <tr>
        <td style="width:100%;border: 1px solid black; border-collapse: collapse"><a href="https://dadosabertos.camara.leg.br/arquivos/votacoesProposicoes/csv/votacoesProposicoes-2020.csv">votacoesProposicoes</a></td>
    </tr>

  <tr>
    <td style="width:100%;border: 1px solid black; border-collapse: collapse"><a href="https://dadosabertos.camara.leg.br/arquivos/proposicoesAutores/csv/proposicoesAutores-2020.csv">proposicoesAutores</a></td>  
    </tr>

</table>

<h3 style="font-style:italic">0.6 - Significado dos sufixos</h3>
<ul>
    <li>Dataframe_F: base proveniente de filtro </li>
    <li>Dataframe_M: base proveniente de pareamento (merge)</li>
    <li>Dataframe_G: base proveniente de agrupamento (grouby)</li>
</ul>

<h2>1 - Importação dos dados</h2>

<h3 style="font-style:italic"><b>1.1 - Carregamento de dados referentes à deputados, proposições, votações e frentes parlamentares</b></h3>

In [ ]:
proposicoes = pd.read_csv("dados-camara/2020/originais/proposicoes-2020.csv", sep=";", encoding="utf-8")
proposicoesAutores = pd.read_csv("dados-camara/2020/originais/proposicoesAutores-2020.csv", sep=";", encoding="utf-8")
proposicoesTemas = pd.read_csv("dados-camara/2020/originais/proposicoesTemas-2020.csv", sep=";", encoding="utf-8")
votacoes = pd.read_csv("dados-camara/2020/originais/votacoes-2020.csv", sep=";", encoding="utf-8")
votacoesOrientacoes = pd.read_csv("dados-camara/2020/originais/votacoesOrientacoes-2020.csv", sep=";", encoding="utf-8")
votacoesProposicoes = pd.read_csv("dados-camara/2020/originais/votacoesProposicoes-2020.csv", sep=";", encoding="utf-8")
votacoesVotos = pd.read_csv("dados-camara/2020/originais/votacoesVotos-2020.csv", sep=";", encoding="utf-8")
frentesDeputados = pd.read_csv("dados-camara/2020/originais/frentesDeputados.csv", sep=";", encoding="utf-8")
deputados =  pd.read_csv("dados-camara/2020/originais/deputados.csv", sep=";", encoding="utf-8")

<h3 style="font-style:italic">1.2 - Listagem de colunas das bases</h3>

In [ ]:
# proposicoes.columns
# list(proposicoesAutores.columns)
list(proposicoesTemas.columns)
# list(votacoes.columns)
# list(votacoesOrientacoes.columns)
# list(votacoesProposicoes.columns)
# list(votacoesVotos.columns)
# list(frentesDeputados.columns)
# list(deputados.columns)     

<h3 style="font-style:italic">1.3 - Remoção de colunas não usadas na análise</h3>

In [ ]:
proposicoes_F1 = proposicoes.drop(['ultimoStatus_dataHora','ultimoStatus_uriRelator','ultimoStatus_idOrgao','ultimoStatus_siglaOrgao','ultimoStatus_uriOrgao','ultimoStatus_regime','ultimoStatus_idSituacao','ultimoStatus_url','codTipo','siglaTipo','ementaDetalhada','dataApresentacao','urlInteiroTeor','ultimoStatus_idTipoTramitacao'], axis = 1)
proposicoesAutores_F1 = proposicoesAutores.drop(['proponente','uriAutor','uriPartidoAutor','uriProposicao','ordemAssinatura'], axis = 1)
proposicoesTemas_F1 = proposicoesTemas2020.drop(['siglaTipo','numero','ano','codTema','relevancia'], axis = 1)
votacoes_F1 = votacoes.drop(['dataHoraRegistro','idOrgao','uriOrgao','siglaOrgao','idEvento','uriEvento','ultimaAberturaVotacao_dataHoraRegistro','ultimaApresentacaoProposicao_dataHoraRegistro','ultimaApresentacaoProposicao_uriProposicao','ultimaApresentacaoProposicao_descricao','ultimaAberturaVotacao_descricao'], axis = 1)
votacoesOrientacoes_F1 = votacoesOrientacoes2020.drop(['uriVotacao','siglaOrgao'], axis = 1)
votacoesProposicoes_F1 = votacoesProposicoes2020.drop(['uriVotacao','proposicao_codTipo'], axis = 1)
votacoesVotos_F1 = votacoesVotos.drop(['uriVotacao','deputado_uri','deputado_uriPartido','deputado_urlFoto','dataHoraVoto'], axis = 1)
frentesDeputados_F1 = frentesDeputados.drop(['deputado_.uri','deputado_.urlFoto','deputado_.codTitulo','dataInicio','dataFim','deputado_.uriPartido','deputado_.siglaUf','deputado_.urlFoto','deputado_.codTitulo','deputado_.uriPartido','deputado_.uri','deputado_.idLegislatura','dataInicio','dataFim','id'], axis = 1)

<h3 style="font-style:italic">1.4 - Arquivamento de bases filtradas</h3>

In [ ]:
proposicoes_F1.to_csv("dados-camara/2020/modificados/proposicoes-2020-F1.csv", sep=";", encoding="utf-8", index=False)
proposicoesAutores_F1.to_csv("dados-camara/2020/modificados/proposicoesAutores-2020-F1.csv", sep=";", encoding="utf-8", index=False)
proposicoesTemas_F1.to_csv("dados-camara/2020/modificados/proposicoesTemas-2020-F1.csv", sep=";", encoding="utf-8", index=False)
votacoes_F1.to_csv("dados-camara/2020/modificados/votacoes-2020-F1.csv", sep=";", encoding="utf-8", index=False)
votacoesOrientacoes_F1.to_csv("dados-camara/2020/modificados/votacoesOrientacoes-2020-F1.csv", sep=";", encoding="utf-8", index=False)
votacoesProposicoes_F1.to_csv("dados-camara/2020/modificados/votacoesProposicoes-2020-F1.csv", sep=";", encoding="utf-8", index=False)
votacoesVotos_F1.to_csv("dados-camara/2020/modificados/votacoesVotos-2020-F1.csv", sep=";", encoding="utf-8", index=False)
frentesDeputados_F1.to_csv("dados-camara/2020/modificados/frentesDeputados-2020-F1.csv", sep=";", encoding="utf-8", index=False)

<h3 style="font-style:italic">1.5 - Carregamento das bases filtradas</h3>

In [ ]:
proposicoes_F1 = pd.read_csv("dados-camara/2020/modificados/proposicoes-2020-F1.csv", sep=";", encoding="utf-8")
proposicoesAutores_F1 = pd.read_csv("dados-camara/2020/modificados/proposicoesAutores-2020-F1.csv", sep=";", encoding="utf-8")
proposicoesTemas_F1 = pd.read_csv("dados-camara/2020/modificados/proposicoesTemas-2020-F1.csv", sep=";", encoding="utf-8")
votacoes_F1 = pd.read_csv("dados-camara/2020/modificados/votacoes-2020-F1.csv", sep=";", encoding="utf-8")
votacoesOrientacoes_F1 = pd.read_csv("dados-camara/2020/modificados/votacoesOrientacoes-2020-F1.csv", sep=";", encoding="utf-8")
votacoesProposicoes_F1 = pd.read_csv("dados-camara/2020/modificados/votacoesProposicoes-2020-F1.csv", sep=";", encoding="utf-8")
votacoesVotos_F1 = pd.read_csv("dados-camara/2020/modificados/votacoesVotos-2020-F1.csv", sep=";", encoding="utf-8")
frentesDeputados_F1 = pd.read_csv("dados-camara/2020/modificados/frentesDeputados-2020-F1.csv", sep=";", encoding="utf-8")
deputados = pd.read_csv("dados-camara/2020/originais/deputados.csv", sep=";", encoding="utf-8")

<h3 style="font-style:italic">1.6 - Listagem de colunas das bases filtradas</h3>

In [ ]:
list(proposicoes2020_F1.columns)
list(proposicoesAutores2020_F1.columns)
list(proposicoesTemas2020_F1.columns)
list(votacoes2020_F1.columns)
list(votacoesOrientacoes2020_F1.columns)
list(votacoesProposicoes2020_F1.columns)
list(votacoesVotos2020_F1.columns)
list(frentesDeputados_F1.columns)
list(deputados.columns)

<h2>2 - Processamento da base de dados "listaDeputados"</h2>

<h3 style="font-style:italic">2.1 - Filtragem da base do TSE para pegar apenas deputados federais</h3>

In [ ]:
votacao = pd.read_csv("votacao_candidato_munzona_2018_BRASIL.csv", sep=";", encoding="iso-8859-1")
filtroVotacao = votacao[(votacao.DS_CARGO=='Deputado Federal')&(votacao.DS_SIT_TOT_TURNO.str.startswith("ELEITO"))]
filtroVotacao.to_csv("deputados-2018.csv", sep=";", encoding="utf-8")

<h3 style="font-style:italic">2.2 - Agrupamento de informações do TSE para obter registros únicos e números totais de votos</h3>

In [ ]:
votacaoFiltrado = pd.read_csv("deputados-2018.csv", sep=";", encoding="utf-8")
filtroVotacao2 = votacaoFiltrado.groupby(['NM_CANDIDATO']).agg({'NM_URNA_CANDIDATO':'first','SG_PARTIDO':'first', 'DS_COMPOSICAO_COLIGACAO':'first', 'SG_UF':'first', 'NM_MUNICIPIO':'first','QT_VOTOS_NOMINAIS':'sum'})
filtroVotacao2.to_csv("2deputados-2018.csv", sep=";", encoding="utf-8")

<h3 style="font-style:italic">2.3 - Filtragem da base do TSE para pegar apenas deputados federais suplentes</h3>

In [ ]:
votacao = pd.read_csv("votacao_candidato_munzona_2018_BRASIL.csv", sep=";", encoding="iso-8859-1")
filtroVotacao = votacao[(votacao.DS_CARGO=='Deputado Federal')&(votacao.DS_SIT_TOT_TURNO.str.startswith("SUPLENTE"))]
filtroVotacao2 = filtroVotacao.groupby(['NM_CANDIDATO']).agg({'NM_URNA_CANDIDATO':'first','SG_PARTIDO':'first', 'DS_COMPOSICAO_COLIGACAO':'first', 'SG_UF':'first', 'NM_MUNICIPIO':'first','QT_VOTOS_NOMINAIS':'sum'})
filtroVotacao2.to_csv("suplentes-2018.csv", sep=";", encoding="utf-8")

<h3 style="font-style:italic">2.4 - Exportação de informações sobre suplentes que assumiram titularidade</h2>

In [ ]:
efetivados = pd.read_csv("suplentes-tse-2018.csv", sep=";", encoding="utf-8")
filtroEfetivados = efetivados[(efetivados.NM_CANDIDATO.isin(["AELTON JOSÉ DE FREITAS", "BEATRIZ ROSALIA RIBEIRO CAVASSA DE OLIVEIRA", "HILKEA CARLA DE SOUZA MEDEIROS LIMA", "DAVID MICHAEL DOS SANTOS MIRANDA", "RONALDO SANTINI", "FRANCISCO DANILO BASTOS FORTE", "JOSEILDO RIBEIRO RAMOS", "LEONARDO CUNHA DE BRITO", "MARCELO DE BRUM DA COSTA", "MARCOS BEZERRA RIBEIRO SOARES", "PAULO SÉRGIO PARANHOS DE MAGALHÃES", "PEDRO AUGUSTO PALARETI", "EVANDRO ROGERIO ROMAN", "REINHOLD STEPHANES JUNIOR", "JOÃO BATISTA CONTI", "PEDRO TORRES BRANDÃO VILELA", "VIVIANE DA COSTA REIS", "NEUCIMAR FERREIRA FRAGA", "JOSIVALDO DOS SANTOS MELO", "MILTON COELHO DA SILVA NETO"]))]
filtroEfetivados.to_csv("efetivados.csv", encoding='utf-8')

<h3 style="font-style:italic">2.6 - Pareamento entre base de dados e lista da Auditoria Cidadã com função da biblioteca fuzzywuzzy</h3>

In [ ]:
tse = pd.read_csv("deputados-eleitos-tse.csv", sep=",", encoding="utf-8")
auditoria = pd.read_csv("perfis-auditoria-cidada.csv", sep=",", encoding="utf-8")
deputados = pd.read_csv("deputados-camara.csv", sep=",", encoding="utf-8")

tse = tse.sort_values('Nome_Parlamentar')
auditoria = auditoria.sort_values('Nome_Parlamentar')

choices = auditoria['Nome_Parlamentar'].values.tolist()
tse['xNome_Parlamentar'] = tse['Nome_Parlamentar'].apply(lambda x: process.extractOne(x, choices)[0])

tabela1 = tse.merge(auditoria, left_on='xNome_Parlamentar', right_on='Nome_Parlamentar')
tabela2 = pd.merge(tabela1, deputados, how='left', left_on=['NM_CANDIDATO'], right_on=['nomeCivil'])
tabela2.to_csv("pareamento-perfis-5.csv", sep=",", encoding="utf-8")

<h3 style="font-style:italic">2.7 - Busca de perfis oficiais no Twitter</h3>

In [ ]:
count = 5

fileOut = open('results.txt', "w",encoding='utf-8')

with open('deputados.csv', "r",encoding='utf-8') as fileIn: 
    
    nomes = fileIn.readlines()
    for n in nomes:
        # fileOut.write("Gol---"+n)
        usersI = api.search_users(n, count)
        usersM = [user.screen_name for user in usersI]  
        print(str(usersM))
        print(str(usersM), file=fileOut)
        time.sleep(5)

<h3 style="font-style:italic">2.8 - Coleta do número de seguidores e de publicações dos perfis oficiais</h3>

In [ ]:
stats=open("results.txt","w",encoding='utf-8',newline='')

with open('users.txt', "r",encoding='utf-8',newline='') as fileIn: 
    users = fileIn.readlines()
    for user_name in users:
        user = api.get_user(screen_name = user_name)
        bio = str(user.description)
        bio = re.sub(r"\n|\\n|\n\n|\n\n\n"," ",bio)
        bio = re.sub(";",",",bio)
        print(user.screen_name,';',user.id,';',user.followers_count,';',user.statuses_count,';',bio, file=stats)
        print(user.screen_name)
        time.sleep(5)


<h3 style="font-style:italic">2.9 - Pareamento de dados da Câmara e do TSE com os do Twiitter</h3>

In [ ]:
tabela1 = pd.read_csv("db-deputados-1.csv", sep=",", encoding="utf-8")
tse2018 = pd.read_csv("tse-2018.csv", sep=",", encoding="utf-8")
tabela2 = pd.merge(tabela1, tse2018, how='left', left_on=['CPF'], right_on=['NR_CPF_CANDIDATO'])
tabela2.to_csv("db-deputados-2.csv")

<h3 style="font-style:italic">2.10 - Agrupamento e soma de valores referentes a declaração de bens dos deputados em 2018.</h3>
<p>Centavos precisaram ser arredondados para completar esta função</p>

In [ ]:
bensTse2018 = pd.read_csv("tse-bens-2018-1.csv", sep=",", encoding="utf-8")
# bensTse2018.shape[0]
# list(bensTse2018.columns)

bensTse2018['VR_BEM_CANDIDATO'] = bensTse2018['VR_BEM_CANDIDATO'].round(0).astype(int)

somaBens2018 = bensTse2018.groupby(['SQ_CANDIDATO']).agg({'VR_BEM_CANDIDATO':'sum'})
somaBens2018.to_csv("soma-bens-2018.csv")

<h3 style="font-style:italic">2.11 - Pareamento da base de dados com as declarações de bens do TSE</h3>

In [ ]:
tabela1 = pd.read_csv("db-deputados-24-1.csv", sep=",", encoding="utf-8")
somaBens2018 = pd.read_csv("soma-bens-2018.csv", sep=",", encoding="utf-8")

tabela2 = pd.merge(tabela1, somaBens2018, how='left', left_on=['SQ_CANDIDATO'], right_on=['SQ_CANDIDATO'])

tabela2.to_csv("db-deputados-25.csv")

<h3 style="font-style:italic">2.12 - Pareamento da base de dados com lista de ideologia dos partidos</h3>

In [ ]:
tabela1 = pd.read_csv("db-deputados-25.csv", sep=",", encoding="utf-8")
ideologiaPartidos = pd.read_csv("ideologia-partidos.csv", sep=",", encoding="utf-8")

tabela2 = pd.merge(tabela1.astype("str"), ideologiaPartidos.astype("str"), how='left', left_on=['PARTIDO_ATUAL'], right_on=['PARTIDO'])

tabela2.to_csv("db-deputados-26.csv")

<h3 style="font-style:italic">2.13 - Pareamento da base de dados com lista da divisão territorial do Brasil</h3>

In [ ]:
tabela1 = pd.read_csv("db-deputados-26.csv", sep=",", encoding="utf-8")
regioes = pd.read_csv("regioes.csv", sep=",", encoding="utf-8")
tabela2 = pd.merge(tabela1, regioes, how='left', left_on=['ESTADO_ELEICAO'], right_on=['ESTADO'])
tabela2.to_csv("db-deputados-27.csv")

<h2>3 - Processamento da base de dados "proposicoes"</h2>

<h3 style="font-style:italic">3.1 - Pareamento e filtragem das bases de proposições e autores de proposições</h3>

In [ ]:
proposicoes_M = proposicoes_F1.merge(proposicoesAutores2020_F1,how='left',left_on='id', right_on='idProposicao').merge(proposicoesTemas2020_F1,how='left',left_on='id', right_on='idProposicao')
proposicoes_MF = proposicoes_M.drop(['Unnamed: 0_x','Unnamed: 0_y','idProposicao_y','siglaTipo_y','numero_y', 'ano_y','Unnamed: 0','codTema','relevancia','codTipo','uriOrgaoNumerador', 'urnFinal', 'ultimoStatus_dataHora', 'ultimoStatus_sequencia','ultimoStatus_idOrgao', 'ultimoStatus_siglaOrgao','ultimoStatus_uriRelator',  'ultimoStatus_uriOrgao', 'ultimoStatus_regime', 'ultimoStatus_idTipoTramitacao', 'ultimoStatus_idSituacao', 'ultimoStatus_despacho', 'ultimoStatus_url', 'ordemAssinatura','uriAutor','uriPartidoAutor', 'idProposicao_x', 'uriProposicao','codTipoAutor','proponente','relevancia','codTema'], axis=1)

<h3 style="font-style:italic">3.2 - Agrupamento de informações referentes às proposições</h3>

In [ ]:
proposicoesM.shape[0]
unico = proposicoesM['id'].unique()
unico.shape[0]
proposicoes_GMF = proposicoes_MF.astype('str').groupby('id').agg({'uri': 'first','siglaTipo_x': 'first','numero_x':'first','ano_x': 'first','descricaoTipo': 'first','ementa': 'first','ementaDetalhada': 'first','keywords': 'first','dataApresentacao' : 'first','uriPropAnterior': 'first','uriPropPrincipal': 'first','uriPropPosterior': 'first','urlInteiroTeor': 'first','ultimoStatus_descricaoTramitacao':'first','ultimoStatus_descricaoSituacao': 'first','idDeputadoAutor': lambda tags: ','.join(tags),'tipoAutor': 'first','nomeAutor': lambda tags: ','.join(tags),'siglaPartidoAutor': lambda tags: ','.join(tags),'siglaUFAutor': lambda tags: ','.join(tags),'tema':'first'})
# proposicoesG.shape[0]
proposicoes_GMF.to_csv("dados-camara/2020/modificados/proposicoes-G.csv", sep=";", encoding="utf-8")

In [ ]:
proposicoes = pd.read_csv("db-proposicoes.csv")
proposicoes['NOME_AUTOR'] = proposicoes['NOME_AUTOR'].astype("str").apply(lambda x: ','.join(set(x.split(','))))
proposicoes.to_csv("db-proposicoes.csv", sep=";", index=False)

<h2>4 - Processamento das bases de dados "votosDeputados" e "votacoes"</h2>

<h3 style="font-style:italic">4.1 - Agrupamento das bases de orientações e votos a partir de coluna concatenada</h3>

In [ ]:
votacoesOrientacoes_F1['idVotacaoPartido'] =  votacoesOrientacoes_F1['idVotacao'] + votacoesOrientacoes_F1['siglaBancada']
# votacoesOrientacoes['idVotacaoPartido'].unique().shape[0]
votacoesVotos_F1['idVotacaoPartido'] = votacoesVotos_F1['idVotacao'] + votacoesVotos_F1['deputado_siglaPartido']
# votacoesVotos['idVotacaoPartido'].unique().shape[0]
votacoesVotos_F1['idProposicao'] = votacoesVotos_F1['idVotacao'].astype('str').replace("\-(.*)","",regex=True)
votacoesOrientacoes_F1['idProposicao'] = votacoesOrientacoes_F1['idVotacao'].astype('str').replace("\-(.*)","",regex=True)
votacoesProposicoes_F1['idProposicao'] = votacoesProposicoes_F1['idVotacao'].astype('str').replace("\-(.*)","",regex=True)
votosOrientacoes_M = votacoesVotos_F1.merge(votacoesOrientacoes_F1,how='left',on='idVotacaoPartido')
# votosOrientacoes_MF = votosOrientacoes_M.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'idVotacao_y', 'uriVotacao_y', 'deputado_siglaPartido', 'deputado_uriPartido','deputado_idLegislatura', 'deputado_urlFoto','idVotacaoPartido', 'uriBancada','deputado_uri', 'siglaOrgao'],axis=1)
print(votosOrientacoes_M.shape[0],votosOrientacoes_M['idProposicao_x'].unique().shape[0])

votacoesProposicoes_M = votacoes_F1.merge(votacoesProposicoes_F1,how='left',left_on='id', right_on='idVotacao')
votacoes_MF = votacoesProposicoes_M.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'data_y', 'descricao_y', 'dataHoraRegistro', 'idOrgao', 'uriOrgao', 'siglaOrgao', 'idEvento', 'uriEvento','ultimaAberturaVotacao_dataHoraRegistro', 'ultimaAberturaVotacao_descricao', 'ultimaApresentacaoProposicao_dataHoraRegistro', 'ultimaApresentacaoProposicao_descricao', 'ultimaApresentacaoProposicao_idProposicao','ultimaApresentacaoProposicao_uriProposicao','idVotacao', 'uriVotacao','proposicao_codTipo'],axis=1)

<h3 style="font-style:italic">4.2 - Pareamento da base "votosDeputados" com resultadosVotacoes" e deputados</h3>

In [ ]:
v1resultadoVotacoes = pd.read_csv("planilha-google/v0-resultadoVotacoes.csv", sep=",", encoding="utf-8")
v0proposicoes = pd.read_csv("planilha-google/v0-proposicoes.csv", sep=",", encoding="utf-8")
v0deputados = pd.read_csv("planilha-google/v0-deputados.csv", sep=",", encoding="utf-8")
v0votosDeputados = pd.read_csv("planilha-google/v0-0-votosDeputados.csv", sep=",", encoding="utf-8")


v1votosDeputados = v0votosDeputados.merge(v1resultadoVotacoes,how='left',on='ID_VOTACAO')
v1votosDeputados = v1votosDeputados[v1votosDeputados['APROVACAO'].notna()]
v2votosDeputados = v1votosDeputados.astype('str').merge(v0deputados.astype('str'),how='left',left_on='ID_DEPUTADO', right_on='ID_CAMARA')
v2votosDeputados['ID_PROPOSICAO'] = v2votosDeputados['ID_PROPOSICAO'].astype('str').str.replace('.0','')


v3votosDeputados = pd.merge(v2votosDeputados.astype('str'),v0proposicoes[['ID','TEMA','PALAVRAS_CHAVE','EMENTA']].astype('str'), how='left', left_on=['ID_PROPOSICAO'], right_on=['ID'])


v4votosDeputados = v3votosDeputados.drop(['EMENTA_PROPOSICAO','VOTOS_SIM','VOTOS_NAO','VOTOS_OUTROS','DESCRICAO','TIPO_PROPOSICAO','NUMERO_PROPOSICAO','ANO_PROPOSICAO','URL_VOTACAO','URL_PROPOSICAO','DATA_VOTO','NM_COMPLETO','IDADE','COLIGACAO_ELEICAO','EXPERIENCIA_DEP','CIDADE_NASC','ESTADO_NASC','ELEITO_POR','ST_DECLARAR_BENS','TELEFONE','EMAIL','WEBSITE','MIDIAS_SOCIAIS','TEM_TWITTER','LINK_TWITTER','SEGUIDORES_E1','BALANÇO_SEGUIDORES','TWEETS_E1','TWEETS_E2','BALANÇO_TWEETS','ID_TWITTER','SQ_CANDIDATO','NR_TITULO_ELEITORAL','CPF','ID_CAMARA','API_CAMARA','FOTO','LEG_FINAL','LEG_INICIAL','CONDICAO_ELEITORAL','SITUACAO','PARTIDO_ELEICAO','DT_NASCIMENTO','SG_BANCADA'],axis=1)


v5votosDeputados = v4votosDeputados[v4votosDeputados['TEMA'].notna()]

# list(v4votosDeputados.columns)
v5votosDeputados.to_csv("planilha-google/v0-4-votosDeputados.csv", sep=";", encoding="utf-8")
print(v4votosDeputados.shape[0],v4votosDeputados['ID_PROPOSICAO'].unique().shape[0])

In [ ]:
votosDeputados = pd.read_csv("db-votosDeputados.csv")
votosDeputados = votosDeputados[votosDeputados['TEMA'].notna()]
votosDeputados.to_csv("db-votosDeputados.csv", index=False)

<h2>5 - Processamento da base de dados "tweetsDeputados"</h2>

<h3 style="font-style:italic">5.1 - Coleta de publicações dos perfis oficiais</h3>

In [ ]:
sys.path.append("..")

api = tweepy.API(auth, retry_count=5, retry_delay=5, timeout=150, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

def json_format(data_json, current_tweet):
    current_tweet['id'] = data_json.id
    current_tweet['created_at'] = data_json.created_at.strftime('%Y-%m-%dT%H:%M:%SZ')
    current_tweet['username'] = data_json.user.name
    try:
        #media = json.loads(data_json.entities['media'])
        current_tweet['media_type'] = data_json.entities['media'][0]['type']
        current_tweet['has_media'] = 'True'
        #print json.dumps(media)
    except:
        current_tweet['has_media'] = 'False'
        current_tweet['media_type'] = 'None'
    current_tweet['screen_name'] = data_json.user.screen_name
    current_tweet['retweet_count'] = data_json.retweet_count
    current_tweet['favorite_count'] = data_json.favorite_count
    current_tweet['full_text'] = data_json.text
    current_tweet['lang'] = data_json.lang
    return

def gather(api):
    try:
        user_id = api.get_user(perfil).id
    except tweepy.error.TweepError:
        print('User not found.')
        sys.exit(0)

    if perfil:
        fname = perfil + '.json'
        dataset_file = open(fname, 'w', encoding='utf8')
    else:
        fname = perfil + '.json'
        dataset_file = open(fname, 'w', encoding='utf8')

    dataset_file.write('[')
    current_tweet = {}
    dados = api.home_timeline()
    last_id = int(str(json.dumps(dados[0]._json))[55:74])
    counter = 0

    while True:
        try:
            new_tweets = api.user_timeline(id = user_id, count = 200, max_id=str(last_id-1))
            if not new_tweets:
                dataset_file.write(']')
                break
            for tweet in new_tweets:
                if counter != 0:
                    dataset_file.write(',')
                json_format(tweet, current_tweet)
                dataset_file.write(json.dumps(current_tweet, ensure_ascii=False))
                dataset_file.write('\n')
                counter+=1
                if sys.stdout.isatty():
                    print("\rNumber of tweets collected so far...: %i"%counter, end='\r', flush=True)
                else:
                    print(counter, end=' ', flush=True)

            last_id = new_tweets[-1].id
            
        except Exception as e:
            raise

    print("\nAll done. Output written to " + fname)
    dataset_file.close()
    time.sleep(5.5)

with open('perfis-0.csv', "r",encoding='utf-8',newline='') as fileIn: 
    lista_perfis = fileIn.read().split('\n')
    for perfil in lista_perfis:
        perfil = perfil.rstrip()
        gather(api)
        time.sleep(40)

<h3 style="font-style:italic">5.2 - Processamento da primeira coleta de publicações oficiais</h3>

In [ ]:
path_to_json = 'dados-twitter/publicacoes/A-29-12-2020\\' 
json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

dfs = []
# print(chr(92))
for file in file_list:
    data = pd.read_json(file)    
    dfs.append(data)

tweets_c1 = pd.concat(dfs, ignore_index=True)
tweets_c1['coleta'] = "C1"
# tweets_c1.head(5)
# tweets_c1.shape[0]
# tweets_c1['text'].unique().shape[0]
tweetsC1_F1 = tweets_c1[(tweets_c1['created_at'] > '2020-01-01 00:00:00')]
tweetsC1_F1.shape[0]
tweetsC1_F1.to_csv("dados-twitter/tweetsC1_F1.csv", sep=";", encoding="utf-8")

In [ ]:
%%capture
arq = "dados-twitter/tweetsDeputados-F"
arq_csv = arq + ".csv"
arq_grafo = arq + "-grafo.csv"
arq_rt = arq + "-grafo-rt.csv"
arq_mt = arq + "-grafo-mt.csv"
arq_hashtag = arq + "-grafo-hashtags.csv"

outrow = []
with open(arq_csv, "r",encoding='utf-8') as fileIn:  # input file location
    with open(arq_grafo, "w",encoding='utf-8') as fileOut:  # output file location
        writer = csv.writer(fileOut)
        reader = csv.reader(fileIn, delimiter=';')
        writer.writerow(['id','created_at','media_type','retweet_count','favorite_count','lang','coleta','source','target','text'])
        for row in reader:
            idd = row[0]
            created_at = row[1]
            media_type = row[2]
            retweet_count = row[3]
            favorite_count = row[4]
            lang = row[5]
            coleta = row[6]
            screen_name = row[7]
            target = ''
            text = row[8]

            for cell in row:
                target = re.findall(r"@([^\s]+)", cell)
                target = re.sub(r"\[|\'|\:|\]", '', str(target))
                outrow 
            writer.writerow([idd, created_at, media_type, retweet_count, favorite_count, lang, coleta, screen_name, target, text])

df = pd.read_csv(arq_grafo)
# print(df.head(3))
df['target'].replace('', np.nan, inplace=True)
df.dropna(subset=['target'], inplace=True)
df.to_csv(arq_grafo, index=False)

df = pd.read_csv(arq_grafo)
rt = df['text'].str.startswith('RT @')

df[rt].to_csv(arq_rt, index=False)
df2 = pd.read_csv(arq_rt)
df2['target'].replace(r",[\s\S]*$", "", regex=True, inplace= True)
df2.to_csv(arq_rt, index=False)


def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = df['target'].str.split(',').map(len)

res = pd.DataFrame({
                    'source': np.repeat(df['source'], lens),
                    'target': chainer(df['target']),
                    'text': np.repeat(df['text'], lens),
                    'id': np.repeat(df['id'], lens),
                    'created_at': np.repeat(df['created_at'], lens),
                    'media_type': np.repeat(df['media_type'], lens),
                    'retweet_count': np.repeat(df['retweet_count'], lens),
                    'favorite_count': np.repeat(df['favorite_count'], lens),
                    'lang': np.repeat(df['lang'], lens),
                    'coleta': np.repeat(df['coleta'], lens)
                    })

res['target'].replace('', np.nan, inplace=True)
res.dropna(subset=['target'], inplace=True)

mt = ~res['text'].str.startswith('RT @')
res[mt].to_csv(arq_mt, index=False)

outrow = []
with open(arq_csv, "r",encoding='utf-8') as fileIn:  # input file location
    with open(arq_grafo, "w", encoding='utf-8') as fileOut:  # output file location
        writer = csv.writer(fileOut)
        reader = csv.reader(fileIn, delimiter=',')
        writer.writerow(['hashtags'])
        for row in reader:
            # screen_name = row[0]      
            hashtags = ''
            for cell in row:
                hashtags = re.findall(r"#([^\s]+)", cell)
                hashtags = re.sub(r"\[|\'|\:|\]", '', str(hashtags))
                # hashtags = re.sub('['+string.punctuation+']', '', hashtags)
                hashtags = re.sub(r"[^\P{P},]+", "", hashtags)
                hashtags = re.sub(r"^\s|\s|\s\s","",hashtags)
                outrow 
            writer.writerow([hashtags])

dfRW = pd.read_csv(arq_grafo)
dfRW['hashtags'].replace('', np.nan, inplace=True)
dfRW.dropna(subset=['hashtags'], inplace=True)
dfRW.to_csv(arq_grafo, index=False)

with open(arq_grafo, encoding="utf-8") as f:
    vTweets = [row["hashtags"] for row in DictReader(f)]

col1=[]
col2=[]

for tweet in vTweets:
    # print(tweet)
    vPalavras = tweet.split(',')
    nPalavras = len(vPalavras)

    if nPalavras > 2:
        nLinhas = 0
        vNumeros = []
        for i in range(nPalavras):
            b = nPalavras-(i+1)
            if b>0:
                vNumeros.append(b)
            nLinhas = nLinhas + b

        nNumeros= len(vNumeros)
        invNumeros = vNumeros[::-1]
        
        c1 = []
        for i in range(nNumeros):
            for j in range(vNumeros[i]):
                c1.append(vPalavras[i])  
        col1.extend(c1)


        ordemC2 = []
        for i in range(nNumeros):
            for j in invNumeros:
                ordemC2.append(j)
            invNumeros.pop(0)

        Mposicao = []
        posicaoC2 = []
        for i in range(int(nLinhas/nPalavras)):
            for j in range(nPalavras):
                posicaoC2.append(vPalavras[j])
                Mposicao.append(j)

        c2 =[]
        c2[:] = [posicaoC2[i] for i in ordemC2]
        col2.extend(c2)

# usr = pd.DataFrame({'source': np.repeat(df['source'], lens),
# usr = dfRW['usuario'].values[0]
# n = len(col1)
# usrs = [usr]*n

matriz = np.c_[col1,col2]
df = pd.DataFrame(columns=["source","target"], data=matriz)
df['source'].replace('', np.nan, inplace=True)
df.dropna(subset=['source'], inplace=True)
df['target'].replace('', np.nan, inplace=True)
df.dropna(subset=['target'], inplace=True)

df.to_csv(arq_hashtag, index=False)
os.remove(arq_grafo)


In [ ]:
tweetsDeputados = pd.read_csv("dados-twitter/tweetsDeputados-F.csv", sep=";", encoding="utf-8")
tweetsDeputados_T100 = tweetsDeputados.nlargest(100,"retweet_count")
tweetsDeputados_T100S = tweetsDeputados_T100['text']
tweetsDeputados_T100S.head(4)
# tweetsDeputados_T100.shape[0]
tweetsDeputados_T100S.to_csv("dados-twitter/tweetsDeputados-T100S.csv", sep=";", encoding="utf-8", index=False)